In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import precision_score
from sklearn.utils.class_weight import compute_class_weight

In [2]:
!pip3 install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.0 MB/s eta 0:00:00


In [4]:
!gdown --id 1KXxSQvHfw4ho2WW3atcMzI2_9BZJt4b5

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KXxSQvHfw4ho2WW3atcMzI2_9BZJt4b5
To: /content/kddcup.data_10_percent
100% 74.9M/74.9M [00:00<00:00, 78.1MB/s]


In [5]:
#loading the dataset 10 percent of kdd cup99
df_names =['duration',
'protocol_type',
'service',
'flag',
'src_bytes',
'dst_bytes',
'land',
'wrong_fragment',
'urgent',
'hot',
'num_failed_logins',
'logged_in',
'num_compromised',
'root_shell',
'su_attempted',
'num_root',
'num_file_creations',
'num_shells',
'num_access_files',
'num_outbound_cmds' ,
'is_host_login',
'is_guest_login',
'count',
'srv_count',
'serror_rate',
'srv_serror_rate',
'rerror_rate',
'srv_rerror_rate' ,
'same_srv_rate' ,
'diff_srv_rate',
'srv_diff_host_rate' ,
'dst_host_count',
'dst_host_srv_count',
'dst_host_same_srv_rate',
'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate' ,
'dst_host_srv_diff_host_rate',
'dst_host_serror_rate',
'dst_host_srv_serror_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate',
           'prediction']

df = pd.read_csv('/content/kddcup.data_10_percent', header = None)

df.columns = df_names
df.head()
#dropping unused columns :
df.drop(columns = ['duration',
'protocol_type','flag','dst_bytes',
'land',
'wrong_fragment',
'urgent',
'hot',
'num_failed_logins',
'logged_in',
'num_compromised',
'root_shell',
'su_attempted',
'num_root',
'num_file_creations',
'num_shells',
'num_access_files',
'num_outbound_cmds' ,
'is_host_login',
'is_guest_login','serror_rate',
'srv_serror_rate',
'rerror_rate',
'srv_rerror_rate' ,
'same_srv_rate' ,
'diff_srv_rate','dst_host_same_srv_rate',
'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate' ,'dst_host_serror_rate',
'dst_host_srv_serror_rate',
'dst_host_rerror_rate'],inplace=True)
df.sample(20)


,service,src_bytes,count,srv_count,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_srv_diff_host_rate,dst_host_srv_rerror_rate,prediction
195027,ecr_i,1032,509,509,0.00,255,255,0.00,0.0,smurf.
362277,private,0,254,17,0.00,255,17,0.00,0.0,neptune.
14715,http,208,1,3,0.67,7,255,0.02,0.0,normal.
155262,ecr_i,1032,511,511,0.00,255,255,0.00,0.0,smurf.
197365,ecr_i,1032,511,511,0.00,255,255,0.00,0.0,smurf.
386764,private,0,224,20,0.00,255,20,0.00,0.0,neptune.
115669,private,0,245,12,0.00,255,12,0.00,0.0,neptune.
276149,ecr_i,1032,511,511,0.00,255,255,0.00,0.0,smurf.
185088,ecr_i,1032,510,510,0.00,255,255,0.00,0.0,smurf.
424149,ecr_i,520,511,511,0.00,255,255,0.00,0.0,smurf.


In [6]:
# Remove duplicate rows
df_remove = df.drop_duplicates()


In [14]:
X = df_remove.drop(['prediction'],axis = 1)
y = df_remove['prediction']


y.value_counts()


# Filter the DataFrame for rows where service is 'http'
http_df = df_remove[df_remove['service'] == 'ecr_i']

# Group by 'Target' and count the number of samples in each group
http_counts = http_df.groupby('prediction').size()

# Display the result
print(http_counts)

'''http	29232	3089	1827	0.000547345374931582	1827	1827	0.000547345374931582	0'''


prediction
ipsweep.        4
nmap.           1
normal.       177
pod.          203
portsweep.      1
smurf.        614
dtype: int64


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [9]:
train_pool = Pool(data=X_train, label=y_train, cat_features=[0])


In [15]:
# Define the CatBoost model
model = CatBoostClassifier(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    eval_metric='MultiClass',
    custom_metric=['Precision'],
    loss_function='MultiClass',
    random_seed=42,
    verbose = False
)
model.fit(train_pool)



Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Setting Precision metric value to the default 0
Number of the positive class predictions is 0. Set

In [18]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Calculate precision score
precision = precision_score(y_test, y_pred, average='weighted')
print(f'Precision Score: {precision}')

# Print classification report
print(classification_report(y_test, y_pred))

class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
classes=np.unique(y)
# Print each class label and its corresponding weight
for label, weight in zip(classes, class_weights):
    print(f"Class {label}: Weight {weight}")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

           back.       0.99      0.99      0.99       165
buffer_overflow.       1.00      0.50      0.67         6
      ftp_write.       0.00      0.00      0.00         2
   guess_passwd.       0.91      0.91      0.91        11
           imap.       1.00      0.50      0.67         2
        ipsweep.       0.99      0.98      0.99       127
           land.       1.00      1.00      1.00         3
     loadmodule.       0.00      0.00      0.00         2
       multihop.       0.00      0.00      0.00         1
        neptune.       1.00      1.00      1.00      5505
           nmap.       0.91      0.94      0.92        31
         normal.       1.00      1.00      1.00     15138
           perl.       0.00      0.00      0.00         1
            phf.       0.00      0.00      0.00         1
            pod.       1.00      0.98      0.99        41
      portsweep.       0.95      0.51      0.67        39
        rootk

In [19]:
# Access feature importance scores
feature_importances = model.feature_importances_
print("Feature importance scores:", feature_importances)

Feature importance scores: [15.54164891 25.75722799 14.13300946  4.03765423  1.00466453 14.72349318
 18.42492173  2.47482252  3.90255744]


In [20]:
testing = pd.read_csv('features.csv')
testing.iloc[11]
prediction = model.predict(testing)
preds = pd.DataFrame (prediction)
preds.shape
services = testing['service']
preds['service'] = services
pre_csv = preds.to_csv('precsv')


In [ ]:
import pickle
# Save the trained model to a file
with open('catboost_dupremove.pkl', 'wb') as f:
    pickle.dump(model, f)

